In [ ]:
import numpy as np
from scipy.optimize import minimize, Bounds

class TensegrityStructure:
    def __init__(self, node_positions, rod_pairs, cable_pairs, rest_lengths, stiffness, mass, fixed_nodes):
        self.node_positions = node_positions.copy()
        self.rod_pairs = rod_pairs
        self.cable_pairs = cable_pairs
        self.rest_lengths = rest_lengths
        self.stiffness = stiffness
        self.mass = mass
        self.fixed_nodes = fixed_nodes
        self.g = np.array([0, 0, -9.81])

    def pack(self, nodes):
        return nodes.flatten()

    def unpack(self, x):
        return x.reshape(-1, 3)

    def potential_energy(self, x):
        nodes = self.unpack(x)
        P_g = 0
        for k, (i, j) in enumerate(self.rod_pairs):
            r_cm = 0.5 * (nodes[i] + nodes[j])
            P_g += -self.mass[k] * self.g @ r_cm
        P_e = 0
        for k, (i, j) in enumerate(self.cable_pairs):
            L = np.linalg.norm(nodes[i] - nodes[j])
            delta = L - self.rest_lengths[k]
            P_e += 0.5 * self.stiffness[k] * delta ** 2
        return P_g + P_e

    def rod_constraints(self, x):
        nodes = self.unpack(x)
        constraints = []
        for i, j in self.rod_pairs:
            L = np.linalg.norm(nodes[i] - nodes[j])
            constraints.append(L - np.linalg.norm(self.node_positions[i] - self.node_positions[j]))
        for k in self.fixed_nodes:
            constraints.extend((nodes[k] - self.node_positions[k]).tolist())
        return np.array(constraints)

    def ground_constraint(self, x):
        nodes = self.unpack(x)
        return nodes[:, 2]  # z >= 0

    def center_of_mass(self, nodes):
        total_mass = np.sum(self.mass)
        com = np.zeros(3)
        for k, (i, j) in enumerate(self.rod_pairs):
            rod_center = 0.5 * (nodes[i] + nodes[j])
            com += self.mass[k] * rod_center
        return com / total_mass

# --- Trust-constr 前向运动学 ---
def forward_kinematics_trust_verbose_fixed(structure):
    x0 = structure.pack(structure.node_positions)

    eq_constraint = {'type': 'eq', 'fun': structure.rod_constraints}
    ineq_constraint = {'type': 'ineq', 'fun': structure.ground_constraint}
    bounds = Bounds([-np.inf] * len(x0), [np.inf] * len(x0))

    res = minimize(
        fun=structure.potential_energy,
        x0=x0,
        constraints=[eq_constraint, ineq_constraint],
        method='trust-constr',
        bounds=bounds,
        options={
            'maxiter': 10000,
            'gtol': 1e-5,
            'xtol': 1e-6,
            'verbose': 0,
            'disp': True
        }
    )

    if not res.success and "Constraint violation" not in res.message:
        raise RuntimeError(f"Trust-constr optimization failed: {res.message}")
    elif not res.success:
        print(f"⚠️ Warning: Optimization terminated with status: {res.message}")

    return structure.unpack(res.x)

# --- Jacobian 有限差分 ---
def jacobian_fd_trust(structure, q, delta=1e-4):
    n = len(q)
    J = np.zeros((3, n))
    structure.rest_lengths = q
    ne = structure.center_of_mass(forward_kinematics_trust_verbose_fixed(structure))
    for i in range(n):
        dq = np.zeros_like(q)
        dq[i] = delta
        structure.rest_lengths = q + dq
        ne_plus = structure.center_of_mass(forward_kinematics_trust_verbose_fixed(structure))
        J[:, i] = (ne_plus - ne) / delta
    structure.rest_lengths = q
    return J

# --- 逆向运动学主函数 ---
def inverse_kinematics_trust_fully_consistent(structure, q0, ne_target, tol=1e-4, max_iter=10):
    q = q0.copy()
    for _ in range(max_iter):
        structure.rest_lengths = q
        nodes = forward_kinematics_trust_verbose_fixed(structure)
        ne = structure.center_of_mass(nodes)
        error = ne_target - ne
        if np.linalg.norm(error) < tol:
            break
        G = jacobian_fd_trust(structure, q)
        dq = np.linalg.pinv(G) @ error
        q += dq
    return q, forward_kinematics_trust_verbose_fixed(structure)

# --- 主程序（MuJoCo 结构参数建模） ---
if __name__ == "__main__":
    rod_centers = np.array([
        [0.2438013,  -0.23055046,  0.10995744],
        [0.23304155, -0.2781429,   0.0948906],
        [0.24824598, -0.2435365,   0.06010128],
    ])
    z_offset = 0.1625
    node_positions = np.array([
        rod_centers[0] + [0, 0, -z_offset],  # s0
        rod_centers[0] + [0, 0,  z_offset],  # s1
        rod_centers[1] + [0, 0, -z_offset],  # s2
        rod_centers[1] + [0, 0,  z_offset],  # s3
        rod_centers[2] + [0, 0, -z_offset],  # s4
        rod_centers[2] + [0, 0,  z_offset],  # s5
    ])

    rod_pairs = [(0, 1), (2, 3), (4, 5)]
    cable_pairs = [(2, 5), (0, 3), (1, 4)]
    rest_lengths = np.array([0.175, 0.175, 0.175])
    stiffness = np.array([150.0, 150.0, 150.0])
    mass = np.array([0.0418, 0.0418, 0.0418])
    fixed_nodes = [0, 1]

    structure_mj = TensegrityStructure(
        node_positions=node_positions,
        rod_pairs=rod_pairs,
        cable_pairs=cable_pairs,
        rest_lengths=rest_lengths,
        stiffness=stiffness,
        mass=mass,
        fixed_nodes=fixed_nodes
    )

    target_com_mj = np.array([0.25, -0.24, 0.09])
    q0_mj = rest_lengths.copy()

    q_star_mj_final, nodes_final_mj = inverse_kinematics_trust_fully_consistent(
        structure_mj, q0_mj, target_com_mj
    )
    final_com_mj = structure_mj.center_of_mass(nodes_final_mj)

    print("最终重心位置:", final_com_mj)
    print("目标重心位置:", target_com_mj)
    print("重心误差:", np.linalg.norm(final_com_mj - target_com_mj))
    print("最大约束违反:", np.max(np.abs(structure_mj.rod_constraints(structure_mj.pack(nodes_final_mj)))))

Constraint violation exceeds 'gtol'
Number of iterations: 2532, function evaluations: 48640, CG iterations: 4719, optimality: 6.88e-04, constraint violation: 3.15e-02, execution time:  2.7 s.
⚠️ Warning: Optimization terminated with status: Constraint violation exceeds 'gtol'
Constraint violation exceeds 'gtol'
Number of iterations: 2532, function evaluations: 48640, CG iterations: 4719, optimality: 6.88e-04, constraint violation: 3.15e-02, execution time:  2.6 s.
⚠️ Warning: Optimization terminated with status: Constraint violation exceeds 'gtol'
Constraint violation exceeds 'gtol'
Number of iterations: 1019, function evaluations: 20691, CG iterations: 2348, optimality: 7.93e-07, constraint violation: 3.15e-02, execution time:  1.1 s.
⚠️ Warning: Optimization terminated with status: Constraint violation exceeds 'gtol'
Constraint violation exceeds 'gtol'
Number of iterations: 790, function evaluations: 16036, CG iterations: 1860, optimality: 5.02e-07, constraint violation: 3.15e-02, ex

/opt/anaconda3/envs/511proj/lib/python3.10/site-packages/scipy/optimize/_differentiable_functions.py:317: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self.g - self.g_prev)


Constraint violation exceeds 'gtol'
Number of iterations: 1029, function evaluations: 21755, CG iterations: 2732, optimality: 1.09e-05, constraint violation: 3.15e-02, execution time:  1.2 s.
⚠️ Warning: Optimization terminated with status: Constraint violation exceeds 'gtol'
Constraint violation exceeds 'gtol'
Number of iterations: 1029, function evaluations: 21755, CG iterations: 2732, optimality: 1.09e-05, constraint violation: 3.15e-02, execution time:  1.2 s.
⚠️ Warning: Optimization terminated with status: Constraint violation exceeds 'gtol'
Constraint violation exceeds 'gtol'
Number of iterations: 2403, function evaluations: 46189, CG iterations: 3599, optimality: 5.94e-05, constraint violation: 3.15e-02, execution time:  2.5 s.
⚠️ Warning: Optimization terminated with status: Constraint violation exceeds 'gtol'
Constraint violation exceeds 'gtol'
Number of iterations: 559, function evaluations: 11837, CG iterations: 1505, optimality: 9.61e-06, constraint violation: 3.15e-02, ex